In [ ]:
import cv2 as cv
import numpy as np

def img_show(img, win_name="Preview"):
    cv.imshow(win_name, img)
    cv.waitKey(0)
    cv.destroyWindow(win_name)

def calc_occlusion_color(img):
    return tuple(np.mean(img, axis=(0, 1)).round().astype(np.int))

img = cv.imread('tmp.jpg', cv.IMREAD_COLOR)
tuple(np.mean(img, axis=(0, 1)).round().astype(np.int))

In [ ]:
from shapely import geometry
import cv2 as cv
import numpy as np

def mouse_click_event(event, x, y, flags, params):
    if event != cv.EVENT_LBUTTONDOWN:
        return
    
    selected_points = params
    selected_points.append((x, y))

def print_selected_points(selected_points):
    print('\n'.join(map(lambda p: str(p) + ',', selected_points)))

image_file_path = './tmp.jpg'

window_name = "Image preview (q - quit, c - clear, p - print)"
clear_points_key = 'c'
quit_app_key = 'q'
print_points_key = 'p'

selected_points = []

cv.namedWindow(window_name)
cv.setMouseCallback(window_name, mouse_click_event, selected_points)

image_orig = cv.imread(image_file_path)
point_size = int(round(min(image_orig.shape[:2]) * 0.01))

while True:
    curr_image = image_orig.copy()

    if len(selected_points) > 2:
        points_hull = np.squeeze(cv.convexHull(np.asarray(selected_points)))
        print(points_hull.shape)
        cv.drawContours(
            curr_image, [points_hull], 0, color=(127, 127, 127),
            thickness=-1, lineType=cv.LINE_AA
        )

    for point in selected_points:
        cv.circle(curr_image, point, point_size * 2, (0, 0, 0), thickness=-1, lineType=cv.LINE_AA)
        cv.circle(curr_image, point, point_size, (255, 255, 255), thickness=-1, lineType=cv.LINE_AA)
    
    cv.imshow(window_name, curr_image)
    
    key = cv.waitKey(10) & 0xff
    if key == ord(quit_app_key):
        print('exiting...')
        break
    elif key == ord(clear_points_key):
        selected_points.clear()
        print('points cleared...')
    elif key == ord(print_points_key):
        print('selected points:')
        print_selected_points(selected_points)

if cv.getWindowProperty(window_name, cv.WND_PROP_VISIBLE) >= 1:
    cv.destroyWindow(window_name)

# polygon = geometry.Polygon(())

True